In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install --upgrade transformers
!pip install diffusers

In [1]:
import os
import json
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

In [2]:
os.environ["OPENAI_API_KEY"] = "voc-1599600020126677378679667940bb7120f80.30121104"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [3]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.8)
embedding_model = OpenAIEmbeddings()
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
##sd_pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [4]:
prompt_template = """
Generate a realistic real estate listing in the following format:
Neighborhood: <neighborhood name>
Price: <$xxx,xxx>
Bedrooms: <int>
Bathrooms: <int>
House Size: <sqft>

Description: <A paragraph with vivid and enticing details about the house>

Neighborhood Description: <A paragraph that describes the neighborhood and nearby amenities>
"""

In [8]:
listings = [llm(prompt_template).strip() for _ in range(10)]

In [9]:
listings[0]

'Neighborhood: Park Slope, Brooklyn\nPrice: $1,200,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 1,800 sqft\n\nDescription: Welcome to this charming brownstone in the heart of Park Slope. This beautifully renovated home features a spacious living room with original hardwood floors, a cozy fireplace, and large windows that flood the space with natural light. The updated kitchen boasts stainless steel appliances, granite countertops, and a breakfast bar. The master bedroom offers a peaceful retreat with a walk-in closet and en-suite bathroom. Enjoy the convenience of a private backyard oasis perfect for entertaining or relaxing after a long day.\n\nNeighborhood Description: Park Slope is a vibrant neighborhood known for its tree-lined streets, historic brownstones, and abundance of trendy cafes, restaurants, and boutiques. Prospect Park is just a short walk away, offering a picturesque escape from city life with playgrounds, walking paths, and the Prospect Park Zoo. With excellent public t

In [7]:
#For writing from list to .json
with open("listings.json", "w") as f:
    json.dump(listings, f, indent=2)

In [11]:
# Load the JSON file
with open("listings.json", "r") as f:
    listings = json.load(f)

In [12]:
os.makedirs("chromadb", exist_ok=True)
db = Chroma(
    persist_directory="chromadb",
    collection_name="real_estate",
    embedding_function=embedding_model
)
texts = listings
metadatas = [{"id": i} for i in range(len(listings))]
db.add_texts(texts, metadatas=metadatas)
db.persist()

In [5]:
buyer_questions = [
    "How big do you want your house to be?" 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",  
]
answers = [input(q + " ") for q in buyer_questions]
buyer_profile = " ".join(answers)

How big do you want your house to be?What are 3 most important things for you in choosing this property? A comfortable three-bedroom house with a spacious kitchen and a cozy living room. A quiet neighborhood, good local schools, and convenient shopping options. 
Which amenities would you like? A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Which transportation options are important to you? Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.
How urban do you want your neighborhood to be? A balance between suburban tranquility and access to urban amenities like restaurants and theaters.


In [13]:
results = db.similarity_search(buyer_profile, k=3)

In [14]:
personalized = []
for res in results:
    prompt = f"""
You are a helpful real estate assistant.

Buyer's Preferences:
{buyer_profile}

Original Listing:
{res.page_content}

Rewrite the listing description and neighborhood description to subtly emphasize aspects aligned with the buyer's preferences, without changing facts.
"""
    personalized.append(llm(prompt).strip())

In [15]:
personalized

["Updated Listing:\nNeighborhood: Sunset Park\nPrice: $800,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 1,500 sqft\n\nDescription: Step into this inviting 3-bedroom, 2-bathroom home nestled in the sought-after Sunset Park neighborhood. The spacious open concept living area is perfect for gathering with loved ones, featuring a cozy fireplace and plenty of natural light. The modern kitchen boasts stainless steel appliances, granite countertops, and ample space for culinary creations. The master suite offers a walk-in closet for added convenience. Outside, the backyard oasis includes a deck and lush garden, ideal for relaxation and entertaining. With hardwood floors and a two-car garage, this home is move-in ready for those seeking comfort and style.\n\nNeighborhood Description: Sunset Park embodies the perfect blend of tranquility and convenience, with tree-lined streets and vibrant community vibes. Enjoy easy access to local schools, nearby shopping options, and a variety of dining exper